Describe the Structure and Granularity of Each Dataset

Import Required Libraries

In [1]:
# --------------------------------------------------
# Q1–Q3 | Data Understanding, Cleaning, and Alignment
# --------------------------------------------------
import pandas as pd
import numpy as np


Load All CSV Files

In [2]:
# ----------------------
# Load raw homelessness data (CY and 12-month views)
# ----------------------

# Age
age_cy = pd.read_csv("../data/cy_age.csv")
age_12mo = pd.read_csv("../data/12mo_age.csv")

# Race
race_cy = pd.read_csv("../data/cy_race.csv")
race_12mo = pd.read_csv("../data/12mo_race.csv")

# Gender
gender_cy = pd.read_csv("../data/cy_gender.csv")
gender_12mo = pd.read_csv("../data/12mo_gender.csv")


Describe Dataset Structures (Q1)

In [3]:
# ----------------------
# Describe basic structure and dimensions
# ----------------------

def describe_dataset(name, df):
    print(f"📁 {name}")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}\n")

describe_dataset("Age - CY", age_cy)
describe_dataset("Age - 12mo", age_12mo)
describe_dataset("Race - CY", race_cy)
describe_dataset("Race - 12mo", race_12mo)
describe_dataset("Gender - CY", gender_cy)
describe_dataset("Gender - 12mo", gender_12mo)


📁 Age - CY
Shape: (2958, 5)
Columns: ['CALENDAR_YEAR', 'LOCATION_ID', 'LOCATION', 'AGE_GROUP_PUBLIC', 'EXPERIENCING_HOMELESSNESS_CNT']

📁 Age - 12mo
Shape: (374, 5)
Columns: ['CALENDAR_YEAR', 'LOCATION_ID', 'LOCATION', 'AGE_GROUP_PUBLIC', 'EXPERIENCING_HOMELESSNESS_CNT']

📁 Race - CY
Shape: (4362, 5)
Columns: ['CALENDAR_YEAR', 'LOCATION_ID', 'RACE_ETHNICITY', 'ALONE_OR_IN_COMBINATION', 'CNT']

📁 Race - 12mo
Shape: (630, 5)
Columns: ['CALENDAR_YEAR', 'LOCATION_ID', 'GROUPING', 'SUB', 'EXPERIENCING_HOMELESSNESS_CNT']

📁 Gender - CY
Shape: (1738, 4)
Columns: ['CALENDAR_YEAR', 'LOCATION_ID', 'GENDER', 'EXPERIENCING_HOMELESSNESS_CNT']

📁 Gender - 12mo
Shape: (238, 5)
Columns: ['CALENDAR_YEAR', 'LOCATION_ID', 'LOCATION', 'GENDER', 'EXPERIENCING_HOMELESSNESS_CNT']



 Data Dictionary Summary

## ✅ Q1. Dataset Structure & Granularity

All datasets track homelessness data across California counties using a common structure:

| Dataset    | Geographic Unit          | Time Unit     | Demographic Segments                         | Key Fields                           |
| ---------- | ------------------------ | ------------- | -------------------------------------------- | ------------------------------------ |
| **Age**    | LOCATION_ID (CoC region) | CALENDAR_YEAR | AGE_GROUP_PUBLIC (Under 18, 18–24, ..., 65+) | EXPERIENCING_HOMELESSNESS_CNT        |
| **Race**   | LOCATION_ID              | CALENDAR_YEAR | RACE, ALONE_OR_IN_COMBINATION                | CNT or EXPERIENCING_HOMELESSNESS_CNT |
| **Gender** | LOCATION_ID              | CALENDAR_YEAR | GENDER                                       | EXPERIENCING_HOMELESSNESS_CNT        |

- Time span: 2017–2023
- Granularity: County × Year × [Age / Race / Gender]
- 12mo versions represent rolling 12-month estimates.


## 🧼 Q2: Data Cleaning Functions
These functions standardize columns and prepare data for merging.


In [4]:
# ----------------------
# Q2: Cleaning Functions
# ----------------------

def clean_age(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.str.upper().str.strip()
    df = df.rename(columns={'EXPERIENCING_HOMELESSNESS_CNT': 'HOMELESS_COUNT'})
    return df

def clean_race(df: pd.DataFrame, is_cy: bool = True) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.str.upper().str.strip()
    if is_cy:
        df = df.rename(columns={
            'RACE_ETHNICITY': 'RACE',
            'ALONE_OR_IN_COMBINATION': 'RACE_COMBO',
            'CNT': 'HOMELESS_COUNT'
        })
    else:
        df = df.rename(columns={
            'GROUPING': 'RACE',
            'SUB': 'RACE_COMBO',
            'EXPERIENCING_HOMELESSNESS_CNT': 'HOMELESS_COUNT'
        })
    return df

def clean_gender(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.str.upper().str.strip()
    df = df.rename(columns={'EXPERIENCING_HOMELESSNESS_CNT': 'HOMELESS_COUNT'})
    return df


Apply Cleaning (Q2)

In [5]:
# ----------------------
# Apply cleaning to all datasets
# ----------------------

# Age
age_cy_clean = clean_age(age_cy)
age_12mo_clean = clean_age(age_12mo)

# Race
race_cy_clean = clean_race(race_cy, is_cy=True)
race_12mo_clean = clean_race(race_12mo, is_cy=False)

# Gender
gender_cy_clean = clean_gender(gender_cy)
gender_12mo_clean = clean_gender(gender_12mo)


In [6]:
# Show cleaned data sample
age_cy_clean.head()



,CALENDAR_YEAR,LOCATION_ID,LOCATION,AGE_GROUP_PUBLIC,HOMELESS_COUNT
0,2017,All,California,18-24,16275
1,2017,All,California,25-34,27785
2,2017,All,California,35-44,25006
3,2017,All,California,45-54,26669
4,2017,All,California,55-64,23239


## ✅ Q2. Data Cleaning & Standardization Summary

All datasets were cleaned by:
- Renaming inconsistent column names to `HOMELESS_COUNT`
- Standardizing all column headers to uppercase
- Preparing each DataFrame for merging and analysis


Add Unified Identifier (Q3)

In [7]:
# ----------------------
# Q3: Add Unified ID for Merge Compatibility
# ----------------------

def add_unique_id(df: pd.DataFrame) -> pd.DataFrame:
    df['UNIQUE_ID'] = df['LOCATION_ID'].astype(str) + "_" + df['CALENDAR_YEAR'].astype(str)
    return df

# Apply to all cleaned datasets
age_cy_clean = add_unique_id(age_cy_clean)
age_12mo_clean = add_unique_id(age_12mo_clean)
race_cy_clean = add_unique_id(race_cy_clean)
race_12mo_clean = add_unique_id(race_12mo_clean)
gender_cy_clean = add_unique_id(gender_cy_clean)
gender_12mo_clean = add_unique_id(gender_12mo_clean)


Validation of Cleaned Outputs

In [8]:
# ----------------------
# Final integrity check
# ----------------------

cleaned_datasets = {
    "Age CY": age_cy_clean,
    "Race CY": race_cy_clean,
    "Gender CY": gender_cy_clean
}

for name, df in cleaned_datasets.items():
    print(f"\n✅ {name} — Shape: {df.shape}")
    print("Sample UNIQUE_IDs:", df['UNIQUE_ID'].dropna().unique()[:3])



✅ Age CY — Shape: (2958, 6)
Sample UNIQUE_IDs: ['All_2017' 'CA-500_2017' 'CA-501_2017']

✅ Race CY — Shape: (4362, 6)
Sample UNIQUE_IDs: ['All_2017' 'CA-500_2017' 'CA-501_2017']

✅ Gender CY — Shape: (1738, 5)
Sample UNIQUE_IDs: ['All_2017' 'CA-500_2017' 'CA-501_2017']


Save All Cleaned DataFrames 

In [ ]:
import joblib
import os

# Create a folder to store if it doesn't exist
os.makedirs("../data/cleaned", exist_ok=True)

# Save Age datasets
joblib.dump(age_cy_clean, "../data/cleaned/age_cy_clean.pkl")
joblib.dump(age_12mo_clean, "../data/cleaned/age_12mo_clean.pkl")

# Save Race datasets
joblib.dump(race_cy_clean, "../data/cleaned/race_cy_clean.pkl")
joblib.dump(race_12mo_clean, "../data/cleaned/race_12mo_clean.pkl")

# Save Gender datasets
joblib.dump(gender_cy_clean, "../data/cleaned/gender_cy_clean.pkl")
joblib.dump(gender_12mo_clean, "../data/cleaned/gender_12mo_clean.pkl")

print("✅ All cleaned DataFrames saved successfully.")


✅ All cleaned DataFrames saved successfully.
